In [1]:
import os
import numpy as np
import soundfile as sf
from glob import glob

In [2]:
source_dir = '..\\Tho\\From_Tho_playback_speech_tracks\\'
dest_dir = '..\\Tho\\Motu_Ready_Files\\'
gain = 0.08#0.04

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

In [3]:
list = glob(source_dir + '*.npy')
print(list)

for item in list:
    data_raw = np.load(item)
    
    data_play = np.zeros((data_raw.shape[0],10))
    data_play[:,4:10]=data_raw*gain
    sf.write(dest_dir + item.split('\\')[-1].replace('.npy','') + '.wav', data_play, 44100)
    

['..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_1.npy', '..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_10.npy', '..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_11.npy', '..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_12.npy', '..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_2.npy', '..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_3.npy', '..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_4.npy', '..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_5.npy', '..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_6.npy', '..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_7.npy', '..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_8.npy', '..\\Tho\\From_Tho_playback_speech_tracks\\speech_track_9.npy']


---
---
---
---

# Start Here!

In [1]:
import os
import wave
import pyaudio
import time

import socket
from struct import pack, unpack

UDP_IP = '192.168.1.34'
UDP_PORT = 161

In [3]:
def turntable(COMMAND=4, ANGLE=0):
    if COMMAND==3 or COMMAND==4:
        ANGLE=0

    MESSAGE = pack('hh', COMMAND, int(ANGLE*10))

    """
    print("UDP target IP: {}".format(UDP_IP))
    print("UDP target port: {}".format(UDP_PORT))
    print('SENT : {}'.format(MESSAGE))
    """

    sock = socket.socket(socket.AF_INET, # Internet
                        socket.SOCK_DGRAM) #UDP
    sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))
    DATA, ADDR = sock.recvfrom(1024)

    if len(DATA) is not None:
        #print('RECEIVED : {}'.format(DATA))

        if COMMAND==1 or COMMAND==2 or COMMAND==3:
            if DATA[0]==0x33:
                print('UDP COMMAND : SUCCESSFUL')
            elif DATA[0]==0x66:
                print('UDP COMMAND : UNSUCCESSFUL')
            else:
                print('ERROR_1')

        elif COMMAND==4:
            if DATA[0]==0x05:
                READING = unpack('>hh', DATA[1:5])[1]/10
                print('TURNTABLE STOPPED')
                print('CURRENT ANGLE : {}\n'.format(str(READING)))
            elif DATA[0]==0x04:
                print('TURNTABLE STILL IN MOTION')
            else:
                print('ERROR_2')

        else:
            print('ERROR_3')
turntable()

TURNTABLE STOPPED
CURRENT ANGLE : 0.0



In [4]:
def recorder(OUT_DIR,spkr):
    data_play = wave.open('../Tho/Motu_Ready_Files/speech_track_{}.wav'.format(spkr), 'rb')

    p = pyaudio.PyAudio()

    In_Channel = data_play.getnchannels()
    In_Sample = data_play.getsampwidth()
    In_Frames = data_play.getnframes()
    In_Rate = data_play.getframerate()
    In_Duration = In_Frames / float(In_Rate)
    print(In_Rate)

    def callback(in_data, frame_count, time_info, flag):
        data = data_play.readframes(frame_count)
        return (data, pyaudio.paContinue)

    #Start Recording
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 8
    RATE = 44100
    RECORD_SECONDS = In_Duration

    stream_in = p.open(format=FORMAT,
            channels=CHANNELS,
            rate=RATE,
            input=True,
            frames_per_buffer=CHUNK)

    #print('CHANNEL : {}/{}'.format(ch+1, 6))
    #print("RECORDING", end='\n\n')

    frames=[]
    front_buffer = int(RATE/CHUNK*0.5)

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data=stream_in.read(CHUNK)
        frames.append(data)
        #print('.', end='')
        if i == front_buffer:        
            stream_out = p.open(format=p.get_format_from_width(data_play.getsampwidth()),
                    channels=10,
                    rate=RATE,
                    output=True,
                    stream_callback=callback,
                    output_device_index=7)

    # start the stream (4)
    stream_out.start_stream()

    # wait for stream to finish (5)
    while stream_out.is_active():
        time.sleep(0.1)

    # stop stream (6)
    stream_out.stop_stream()
    stream_out.close()
    data_play.close()

    stream_in.stop_stream()
    stream_in.close()

    # close PyAudio (7)
    p.terminate()

    #data_rec = wave.open(OUT_DIR + 'RIR_' + str(ch+1) +'.wav', 'wb')
    data_rec = wave.open(OUT_DIR + 'speech_track_{}_recorded'.format(spkr) +'.wav', 'wb')
    data_rec.setnchannels(CHANNELS)
    data_rec.setsampwidth(p.get_sample_size(FORMAT))
    data_rec.setframerate(RATE)
    data_rec.writeframes(b''.join(frames))
    data_rec.close()


In [5]:
import time, datetime

# Change Elev

In [6]:
start = time.time()

#ANGLE_LIST=[0, 11, 22, 33, 44, 55]

ELEVATION = 0 # 0, 1000

for ANGLE in range(0, 60, 5):

    a_str = time.time()
    
    #OUT_DIR= '..\\Tho\\Recorded_Tracks\\ELEV_' + str(ELEVATION) + '_PADDED\\' + 'ANG_' + str(ANGLE) + '\\'
    OUT_DIR= '..\\Tho\\Recorded_Tracks\\ELEV_' + str(ELEVATION) + '_SPEECH\\' + 'ANG_' + str(ANGLE) + '\\'
    
    spkr=int(ANGLE/5+1)

    print(OUT_DIR)
    if not os.path.exists(OUT_DIR):
        os.makedirs(OUT_DIR)
        
    turntable()
    recorder(OUT_DIR, spkr)
    turntable(1,5)
    time.sleep(3)
    
    a_end = time.time()
    print("Time Elapsed for ANG_{}: {} ".format(ANGLE,datetime.timedelta(0, a_end-a_str)))

end = time.time()
print("Total Time Elapsed : {} ".format(datetime.timedelta(0, end-start)))

..\Tho\Recorded_Tracks\ELEV_0_SPEECH\ANG_0\
TURNTABLE STOPPED
CURRENT ANGLE : 0.0

44100
UDP COMMAND : SUCCESSFUL
Time Elapsed for ANG_0: 0:06:44.891568 
..\Tho\Recorded_Tracks\ELEV_0_SPEECH\ANG_5\
TURNTABLE STOPPED
CURRENT ANGLE : 5.0

44100
UDP COMMAND : SUCCESSFUL
Time Elapsed for ANG_5: 0:06:44.725352 
..\Tho\Recorded_Tracks\ELEV_0_SPEECH\ANG_10\
TURNTABLE STOPPED
CURRENT ANGLE : 10.0

44100
UDP COMMAND : SUCCESSFUL
Time Elapsed for ANG_10: 0:06:44.687950 
..\Tho\Recorded_Tracks\ELEV_0_SPEECH\ANG_15\
TURNTABLE STOPPED
CURRENT ANGLE : 15.0

44100
UDP COMMAND : SUCCESSFUL
Time Elapsed for ANG_15: 0:06:44.725798 
..\Tho\Recorded_Tracks\ELEV_0_SPEECH\ANG_20\
TURNTABLE STOPPED
CURRENT ANGLE : 20.0

44100
UDP COMMAND : SUCCESSFUL
Time Elapsed for ANG_20: 0:06:44.811397 
..\Tho\Recorded_Tracks\ELEV_0_SPEECH\ANG_25\
TURNTABLE STOPPED
CURRENT ANGLE : 25.0

44100
UDP COMMAND : SUCCESSFUL
Time Elapsed for ANG_25: 0:06:44.758968 
..\Tho\Recorded_Tracks\ELEV_0_SPEECH\ANG_30\
TURNTABLE STOPPED
C

OSError: [Errno 28] No space left on device

# END!